# Packages

In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, DoubleType, ByteType, ShortType, StringType, DecimalType, LongType, BinaryType
from pyspark.sql.functions import col, udf
from shapely.geometry import Polygon
from shapely import wkt
from pyproj import Transformer
# Legger på nødvendige metadata og tar i bruk DASK sin metadata-validering  
from dask_felleskomponenter.governance.main import Metadata


# Constant variable

In [0]:
BUCKET_FOLDER = "/AR50/"

# Config

In [0]:
%run ./config

{'catalog_name': 'land_techtroll_dev',
 'landing_zone_prefix': '/Volumes/land_techtroll_dev/external_dev/landing_zone',
 'location_prefix': '/Volumes/land_techtroll_dev/external_dev/static_data/cloudFiles',
 'static_data_prefix': '/Volumes/land_techtroll_dev/external_dev/static_data',
 'env': 'dev'}

# Drop tabel
Hvis kan endre strukturen eller partitionering, så er det greit å droppe tabellen for å unngå error meldinger. Dette går greit siden den statiske dataen som blir lest inn til databasen endrer seg ikke

In [0]:

%sql
DROP TABLE IF EXISTS bronze.ar50;
     

# Setup of catelog

In [0]:
spark.sql(f'USE CATALOG {spark.conf.get("conf.catalog_name")}')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS bronze')
spark.sql('USE SCHEMA bronze')

DataFrame[]

# Read bucket
Leser data som satt inn i bucket hos google cloud.


In [0]:
landing_zone = spark.conf.get("conf.landing_zone_prefix")
ar50_path = landing_zone + "/AR50"

In [0]:
df = (spark.read
    .format("xml")
    .option("rowTag", "app:ArealressursFlate")     # arealressursflate e ei lag i hope, i believe, i wish, i beg
    .load(ar50_path)
)


In [0]:


df = df.select(
    col("`_gml:id`").alias("gml_Id"),
    col("`app:arealtype`").alias("Arealtype"),
    col("`app:identifikasjon`.`app:Identifikasjon`.`app:lokalId`").alias("lokalId"),
    col("`app:identifikasjon`.`app:Identifikasjon`.`app:navnerom`").alias("Navnerom"),
    col("`app:informasjon`").alias("Informasjon"),
    col("`app:jordbruk`").alias("Jordbruk"),
    col("`app:kartstandard`").alias("Kartstandard"),
    col("`app:kopidata`.`app:Kopidata`.`app:kopidato`").alias("Kopiddato"),
    col("`app:kopidata`.`app:Kopidata`.`app:områdeId`").alias("områdeId"),
    col("`app:kopidata`.`app:Kopidata`.`app:originalDatavert`").alias("OriginalDatavert"),
    col("`app:oppdateringsdato`").alias("Oppdateringsdato"),
    col("`app:skogbonitet`").alias("Skogbonitet"),
    col("`app:treslag`").alias("Treslag"),
    col("`app:vegetasjonsdekke`").alias("vegetasjonsdekke"),
    # Her trekker vi ut posList (koordinatene) fra geometri (exterior). 
    col("`app:område`.`gml:Surface`.`gml:patches`.`gml:PolygonPatch`.`gml:exterior`.`gml:LinearRing`.`gml:posList`").alias("geometry")
)


In [0]:
# Transformer fra UTM 33 (EPSG:25833) til WGS84
transformer = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)

# UDF som tar posList-streng og returnerer WKT i EPSG:4326
def poslist_to_wkt(poslist):
    try:
        coords = list(map(float, poslist.strip().split()))
        points = list(zip(coords[::2], coords[1::2]))

        if points[0] != points[-1]:
            points.append(points[0])

        polygon = Polygon(points)

        if not polygon.is_valid:
            polygon = polygon.buffer(0)

        if not polygon.is_valid:
            return None
        
        transformed = [transformer.transform(x, y) for x, y in polygon.exterior.coords]
        new_poly = Polygon(transformed)

        if new_poly.is_valid:
            return new_poly.wkt
        else:
            return None
    except Exception:
        return None


In [0]:
poslist_to_wkt_udf = udf(poslist_to_wkt, StringType())

df = df.withColumn("geometry", poslist_to_wkt_udf("geometry"))


In [0]:
# Skriver ut data for å se lastet inn riktig
df.select("geometry").display(10)

geometry
"POLYGON ((8.744245657602976 58.79385284164507, 8.743951382318008 58.7940411530512, 8.743771324358187 58.79407741936443, 8.743607393775658 58.79402441122858, 8.743435399651537 58.79401603997448, 8.743255341331393 58.794052305558345, 8.743120957312465 58.79431594205738, 8.743032245642343 58.79480694957497, 8.742897856638868 58.795070585746586, 8.742717792254515 58.79510685046878, 8.742658661628884 58.79519403137258, 8.742720465377827 58.795332128635394, 8.742892465776997 58.79534050095698, 8.743174661009983 58.79521914786289, 8.743354725549391 58.795182882503354, 8.743432661031466 58.79523170533688, 8.74365169330487 58.79521985096007, 8.74401182111758 58.795147318629304, 8.744273852694064 58.79513755584781, 8.744437789505104 58.79519056324171, 8.744452568580204 58.79534888583142, 8.744318187911858 58.79561252363139, 8.74416096349069 58.79576247578481, 8.743980896669745 58.79579874223322, 8.743878767020453 58.79588383102109, 8.743854574649477 58.796017742335835, 8.743799477273646 58.79608260523114, 8.743713475076817 58.796078419670906, 8.74370541070373 58.79612305676137, 8.743775284408128 58.7962165165214, 8.744330269601132 58.796266039984204, 8.745370366675683 58.79627162087156, 8.745984480092009 58.796233954998826, 8.746172608530811 58.79615304845001, 8.746262641873042 58.796134913720756, 8.746254580754892 58.79617955102855, 8.746379554575531 58.7962081454731, 8.746637563518755 58.796220696669494, 8.746875421794758 58.79634484075283, 8.747093131398758 58.796580577869086, 8.74741699808896 58.796708904356656, 8.747847020164084 58.79672981938411, 8.748066060922037 58.79671795763326, 8.748074119885112 58.79667332017453, 8.748207155864005 58.79665727543829, 8.748465169042433 58.7966698230398, 8.74868823835552 58.79663564147013, 8.748876362983324 58.79655473089667, 8.749142433274688 58.79652263959591, 8.749486450178106 58.79653936710055, 8.749928556655215 58.79649331876878, 8.750468749962407 58.79638449353949, 8.75097802070927 58.79620661904173, 8.75145636623424 58.795959695595066, 8.751703590464865 58.79579159538342, 8.75172775238927 58.79565768205323, 8.751082734609692 58.795626328721816, 8.75083278295478 58.7955691487875, 8.750762891158109 58.79547569254413, 8.750594915165514 58.79544501185853, 8.75032885436084 58.79547710633528, 8.750160878782864 58.795446425089594, 8.750090989047754 58.79535296851685, 8.749591093997333 58.79523860444873, 8.748661199108255 58.79510332796326, 8.748001400049672 58.79491363701858, 8.747611695163272 58.79466953574498, 8.747197817623885 58.79455934558602, 8.746759762963803 58.794583066230416, 8.746536705311822 58.794617244660536, 8.746528645026702 58.79466188200233, 8.746309616954964 58.794673741357414, 8.745879621574359 58.79465282165556, 8.745395874858962 58.79444916715225, 8.744858385236252 58.794062777385946, 8.744503648015156 58.79386539679937, 8.744245657602976 58.79385284164507))"
"POLYGON ((8.750451216868196 58.81522840807093, 8.750826353282514 58.81531418253704, 8.751103348882125 58.815462730855295, 8.751407251398607 58.815702644500774, 8.751645256574337 58.815826782272296, 8.751817361371108 58.815835144483614, 8.751977376238093 58.8159104629555, 8.752125301993036 58.816052737747, 8.752285318529324 58.81612805586446, 8.75245742502923 58.81613641724876, 8.75273846378984 58.81626264342722, 8.753128438834393 58.81650673375495, 8.753534535938972 58.81666154742317, 8.753956752267293 58.816727084230465, 8.754218946039623 58.816717304035166, 8.754321115612564 58.81663220778732, 8.754458254931983 58.816593838979365, 8.754630364454279 58.816602197507606, 8.7547903891037 58.816677512648376, 8.754938329698097 58.816819784460755, 8.755098356016925 58.81689509924708, 8.755356522891745 58.81690763571299, 8.755392774693782 58.81670676497239, 8.755541996742366 58.816601438071025, 8.755816272300633 58.81652469764962, 8.755961464732444 58.81644168921713, 8.755977574855699 58.81635241322524, 8.756122766163388 58.816269404590315, 8.756397037837646 58.81619266286061, 8.756546254590235 58.81608733469864,

# Setup for database (delta lake)
Oppsett følger Medaljongarkitektur. Koden støtter hovedsaklig bare bronze data, altså den ubehandlet data leser inn ifra bucket til database. Vurdering om å lage sølv tabell kan vurders

Relevante lenker:
- https://www.databricks.com/glossary/medallion-architecture


In [0]:
# Skriver bronze ut data til databasen uten partitionering
df.write.mode("overwrite").saveAsTable("bronze.ar50")

# Setup for metadata
Sett tags for å beskrive tabell som blir laget


In [0]:
%sql
ALTER TABLE land_techtroll_dev.bronze.tilsyn SET TAGS ( 'medaljongnivaa' = 'bronze');
COMMENT ON TABLE land_techtroll_dev.bronze.tilsyn IS 'Midlertidlig AR50 i databaricks';
ALTER TABLE land_techtroll_dev.bronze.tilsyn SET TAGS ( 'tilgangsnivaa' = 'Ugradert');

In [0]:
catalog_name = spark.conf.get('conf.catalog_name')
# Sjekker at tabellen tilfredsstiller kravene for bronse-produkter
metadata = Metadata(catalog=catalog_name, schema='bronze', table='ar50')
metadata.validate()

[MetadataError(catalog='land_techtroll_dev', schema='bronze', table='ar50', column=None, description="🔴 Feil: 'medaljongnivaa' mangler i table properties. Type: <valør> - gyldige verdier: ['bronze', 'silver', 'gold']", solution="ALTER TABLE land_techtroll_dev.bronze.ar50 SET TAGS ( 'medaljongnivaa' = '<<SETT_MEDALJONGNIVAA_HER>>')", for_field='medaljongnivaa', valid_values=['bronze', 'silver', 'gold'])]